In [1]:
import pandas as pd
import pymysql.cursors
import pymysql

In [2]:
# Conexión a un archivo CSV preprocesado perteneciente a la DB MIMIC-III
# Lectura del archivo CSV en un DataFrame de Pandas
chartevents = pd.read_csv(r'C:\Users\rocio\OneDrive\Escritorio\Cuidados_Intensivos\Modelos_Machine_Learning\ETL_Carli\chartevents.csv')

In [3]:
chartevents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668862 entries, 0 to 668861
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   subject_id    668862 non-null  int64  
 1   hadm_id       668862 non-null  int64  
 2   stay_id       668862 non-null  int64  
 3   caregiver_id  644622 non-null  float64
 4   charttime     668862 non-null  object 
 5   storetime     667703 non-null  object 
 6   itemid        668862 non-null  int64  
 7   value         668861 non-null  object 
 8   valuenum      257474 non-null  float64
 9   valueuom      162571 non-null  object 
 10  warning       667703 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 56.1+ MB


In [4]:
chartevents.columns= chartevents.columns.str.lower()

In [5]:
chartevents.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:45:00,225054,On,NaN,NaN,0.0
1,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:43:00,223769,100,100.0,%,0.0
2,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:47:00,223956,Atrial demand,NaN,NaN,0.0
3,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:47:00,224866,Yes,NaN,NaN,0.0
4,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:45:00,227341,No,0.0,NaN,0.0


In [6]:
chartevents['itemid'].unique()

array([225054, 223769, 223956, ..., 223766, 226096, 228764], dtype=int64)

In [7]:
diccionario = pd.read_csv(r"C:\Users\rocio\OneDrive\Escritorio\Cuidados_Intensivos\Modelos_Machine_Learning\ETL_Carli\d_items.csv")
diccionario.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,226228,Gender,Gender,chartevents,ADT,NaN,Text,NaN,NaN
1,226545,Race,Race,chartevents,ADT,NaN,Text,NaN,NaN
2,229877,Suction events (CH),Suction events (CH),chartevents,ECMO,NaN,Text,NaN,NaN
3,229875,Oxygenator visible (CH),Oxygenator visible (CH),chartevents,ECMO,NaN,Text,NaN,NaN
4,229266,Cannula sites visually inspected (ECMO),Cannula sites visually inspected (ECMO),chartevents,ECMO,NaN,Text,NaN,NaN


In [9]:
diccionario= diccionario.drop(['abbreviation', 'linksto', 'category', 'unitname', 'param_type'], axis=1)

In [10]:
chartevents = pd.merge(chartevents, diccionario, on= 'itemid') 

In [11]:
chartevents.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,lownormalvalue,highnormalvalue
0,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:45:00,225054,On,NaN,NaN,0.0,Anti Embolic Device Status,NaN,NaN
1,10005817,20626031,32604416,4793.0,2132-12-16 20:00:00,2132-12-17 06:20:00,225054,On,NaN,NaN,0.0,Anti Embolic Device Status,NaN,NaN
2,10005817,20626031,32604416,4793.0,2132-12-17 00:00:00,2132-12-17 06:21:00,225054,On,NaN,NaN,0.0,Anti Embolic Device Status,NaN,NaN
3,10005817,20626031,32604416,20310.0,2132-12-17 08:00:00,2132-12-17 07:56:00,225054,On,NaN,NaN,0.0,Anti Embolic Device Status,NaN,NaN
4,10005817,20626031,32604416,20310.0,2132-12-17 12:00:00,2132-12-17 13:48:00,225054,On,NaN,NaN,0.0,Anti Embolic Device Status,NaN,NaN


In [12]:
#Creo una lista con los labs que necesito para mi modelo de ML
analisis_ml = [50821, 51265, 50885, 220052, 198, 50912, 40055, 618, 855, 220050, 211, 51, 223761, 3395, 40055, 51006, 51300, 50822, 50824, 50882, 50885]

In [13]:
# creamos un nuevo DataFrame que contenga solo las filas que contienen elementos de la lista
chartevents_ml = chartevents[chartevents['itemid'].isin(analisis_ml)]

In [14]:
chartevents_ml.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,lownormalvalue,highnormalvalue
132908,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-16 00:02:00,220052,58,58.0,mmHg,0.0,Arterial Blood Pressure mean,NaN,NaN
132909,10005817,20626031,32604416,6770.0,2132-12-16 01:00:00,2132-12-16 01:04:00,220052,63,63.0,mmHg,0.0,Arterial Blood Pressure mean,NaN,NaN
132910,10005817,20626031,32604416,6770.0,2132-12-16 02:00:00,2132-12-16 02:11:00,220052,58,58.0,mmHg,0.0,Arterial Blood Pressure mean,NaN,NaN
132911,10005817,20626031,32604416,6770.0,2132-12-16 03:00:00,2132-12-16 03:00:00,220052,59,59.0,mmHg,0.0,Arterial Blood Pressure mean,NaN,NaN
132912,10005817,20626031,32604416,6770.0,2132-12-16 04:00:00,2132-12-16 04:21:00,220052,66,66.0,mmHg,0.0,Arterial Blood Pressure mean,NaN,NaN


In [16]:
chartevents_ml.to_csv('chartevents_ml_2.csv', index=False)
